# Required Imports

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import json
import re
import torch
from datetime import datetime

/home/pranav/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/pranav/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:758: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [4]:
! pip install datasets
! pip install transformers[torch]
! pip install tokenizers
! pip install evaluate
! pip install rouge_score
! pip install sentencepiece
! pip install huggingface_hub
!pip install timexy

Defaulting to user installation because normal site-packages is not writeable
zsh:1: no matches found: transformers[torch]
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import nltk
import evaluate
import datasets
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq,TFMT5ForConditionalGeneration, MT5Tokenizer, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
from datasets import Dataset, DatasetDict
import timexy
from timexy import Timexy
from datasets import Dataset, DatasetDict
from timexy import rule
from timexy.languages import en
# import tensorflow
# from tensorflow.keras.optimizers import Adam

2024-04-21 21:50:03.152740: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 21:50:03.196097: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 21:50:04.141242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
nlp = spacy.load("en_core_web_sm")

# Optionally add config if varying from default values
config = {
    "kb_id_type": "timex3",  # possible values: 'timex3'(default), 'timestamp'
    "label": "timexy",       # default: 'timexy'
    "overwrite": False       # default: False
}
nlp.add_pipe("timexy", config=config, before="ner")

# Model Preparation

In [ ]:
# Run it the first time
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

chkpnt = './input/checkpoints/chkpnt1'
model.save_pretrained(chkpnt)
tokenizer.save_pretrained(chkpnt)

In [8]:
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 6
drop_out = 0.1

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(chkpnt)
tokenizer = T5Tokenizer.from_pretrained(chkpnt)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
model.config.dropout_rate = drop_out

In [9]:
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
  #  save_steps=num_train_steps_per_epoch,
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [10]:
def preprocess_fun(batch):
  prefix = "Generate: "
  clean_batch=[]
  check=[]
  for i in batch['Inputs']:
    x=""
    l=[]
    for j in i.split('\n')[:-1]:
      x += re.split('~__(True|False)__',j)[0]
      l.append(re.split('~__(True|False)__',j)[1])
    clean_batch.append(x)
    check.append(l)
  # print(clean_batch)
  # The "inputs" are the tokenized answer:
  inputs = [(prefix + doc) for doc in clean_batch]
  #print(inputs)
  model_inputs = tokenizer(inputs, max_length=512, truncation=True,padding='max_length')
  # print(model_inputs.keys())
  # The "labels" are the tokenized outputs:
  labels = tokenizer(text_target=batch["Outputs"],
                      max_length=512,
                      truncation=True,
                     padding='max_length')
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [17]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [13]:
# def compute_metrics(eval_preds):
#     preds,refs = eval_preds

#     decode_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(refs, skip_special_tokens=True)

#     pred_tags = [nlp(seq) for seq in decode_preds]
#     actual_tags = [nlp(seq) for seq in decoded_labels]

#     # Log Likelihood
#     conti = nltk.ContigencyMeasures()
#     ct = conti.contigency_table(actual_tags,pred_tags)
#     err = nltk.metrics.scores.log_likelihood(ct)
#     result = err

#     # Precision
#     # prec = nltk.metrics.scores.precision(actual_tags,pred_tags)

#     # Recall
#     # rec = nltk.metrics.scores.recall(actual_tags,pred_tags)

#     # F1
#     # f1 = 2/(prec + rec)
#     # result = f1

#     # accuracy
#     # result = nltk.metrics.scores.accuracy(actual_tags,pred_tags)

#     #result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

#     return result

In [11]:
def compute_metrics(eval_preds):
    preds,refs = eval_preds

    print(preds)
    if preds.ndim == 0:
        preds = preds.unsqueeze(0)

    if refs.ndim == 0:
        refs = refs.unsqueeze(0)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # pred_tags = [nlp(seq) for seq in decode_preds]
    # actual_tags = [nlp(seq) for seq in decoded_labels]
    # err = nltk.metrics.scores.log_likelihood(actual_tags,pred_tags)
    # nltk.download("punkt", quiet=True)
    # metric = evaluate.load("rouge")
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return result

# Dataset Preparation

In [41]:
df = pd.read_csv('./dataset.csv')
data = Dataset.from_pandas(df)
perc = 80
train_size = int((perc/100)*max(df['Chunk']))

In [43]:
c=0
for index, row in df.iterrows():
    if((row['Chunk'])<=train_size):
        c+=1
train_main = (df.iloc[:c,:])
# valid = (df.iloc[c:v+c,:]).sample(frac=1)

# test (rest)
test = (df.iloc[c:,:])
tsd = Dataset.from_pandas(test)

In [ ]:
# tokenized_dataset.save_to_disk('tokenized_dataset')
# !zip -r tokenized_dataset.zip tokenized_dataset
# from google.colab import files
# files.download('tokenized_dataset.zip')

# Training

In [63]:
def crossValid(k=10):
#   flag = 0
  sm = int(np.floor(train_size/k)) # each batch size
  split_data = [(i*sm,(i+1)*sm) for i in range(k-1)] + [((k-1)*sm,train_size)]
  evals = [] # model evaluations and model

  for split in split_data:
    t_i,v_i = [],[]
    for index,row in train_main.iterrows():
        if(split[0]<=row['Chunk']<=split[1]):
            v_i.append(index)
        else:
            t_i.append(index)
    
    valid = df.iloc[v_i]
    train = df.iloc[t_i]
#     print(f"Here: {max(train['Chunk'])}")
    # Get unique chunk identifiers
#     unique_chunks = train["Chunk"].unique()

# Shuffle the unique chunks
#     np.random.shuffle(unique_chunks)

# Reorder the DataFrame based on the shuffled chunks
#     train = pd.concat([train[train["Chunk"] == chunk] for chunk in unique_chunks], ignore_index=True)
    
    
#     unique_chunks = valid["Chunk"].unique()

# Shuffle the unique chunks
#     np.random.shuffle(unique_chunks)

# Reorder the DataFrame based on the shuffled chunks
#     valid = pd.concat([valid[valid["Chunk"] == chunk] for chunk in unique_chunks], ignore_index=True)

#     for index,rows in valid.iterrows():
        
#     for index,rows in train.iterrows():
#     print(flag,max(train["Chunk"]),min(train["Chunk"]))
#     print(valid)
#     flag+=1
    trd = Dataset.from_pandas(train)
    vdd = Dataset.from_pandas(valid)

    dataset_dict = DatasetDict(
      {
          'train':trd,
          'valid':vdd,
          'test':tsd
      }
    )

    tokenized_dataset = dataset_dict.map(preprocess_fun,batched=True)

    trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )
    trainer.train()
    evals.append((trainer.evaluate(),model))
  return evals

In [ ]:
# tokenized_dataset = datasets.load_from_disk("/content/drive/MyDrive/tokenized_dataset")

In [30]:
evals = crossValid()

ValueError: No objects to concatenate

# Testing

In [23]:
trainer.predict(tokenized_dataset["test"])

/home/pranav/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

# Saving

In [35]:
c = datetime.now()
s1 = f"./savepoints_{c}/model"
s2 = f"./savepoints_{c}/tokenizer"
model.save_pretrained(s1)
tokenizer.save_pretrained(s2)

('./savepoints_2024-04-20 09:55:49.832107/tokenizer/tokenizer_config.json',
 './savepoints_2024-04-20 09:55:49.832107/tokenizer/special_tokens_map.json',
 './savepoints_2024-04-20 09:55:49.832107/tokenizer/spiece.model',
 './savepoints_2024-04-20 09:55:49.832107/tokenizer/added_tokens.json')

In [12]:
model = T5ForConditionalGeneration.from_pretrained("./savepoints_2024-04-21 20:23:55.826915/model")
tokenizer = T5Tokenizer.from_pretrained("./savepoints_2024-04-21 20:23:55.826915/tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Manual Testing

In [6]:
# Example text for generation
text = '''
Generate:
How many days in a week?'''

# Tokenize the text and generate output
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)

# Decode the output tokens to text
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated output:", result)

NameError: name 'tokenizer' is not defined